In [ ]:
# 🧠 Clasificación de Cáncer de Mama con CNNs
# Proyecto: Breast Histopathology Images (IDC)

# ---------------------------------------------
# ⚙️ Setup inicial en Google Colab
# ---------------------------------------------

# Si es tu primera vez:
# !apt-get install git

# 🔁 Clonar repositorio del curso
!git clone https://github.com/solivare/DeepNeuralNetworkUSS.git

# Cambiar al directorio del proyecto
%cd DeepNeuralNetworkUSS/Examples/BreastCancer/

# Instalar dependencias
!pip install -r requirements.txt

In [ ]:
# ⚙️ Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📁 Verificar contenido
!ls "/content/drive/MyDrive/DeepNeuralNetworkUSS/Data/breast_images_prepared"

In [ ]:
# ---------------------------------------------
# 🛠️ Ajuste de rutas de importación
# ---------------------------------------------

import os
import sys

src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.append(src_path)

In [ ]:
# ---------------------------------------------
# 🧼 Renombrar directorio de datos preprocesados
# ---------------------------------------------

import train

# 🔁 Sobrescribir la ruta de los datos con la de Drive
config = train.load_config("config.yaml")
config["paths"]["prepared_data"] = "/content/drive/MyDrive/DeepNeuralNetworkUSS/Data/breast_images_prepared"

In [ ]:
# ---------------------------------------------
# 🧠 Entrenamiento del modelo
# ---------------------------------------------

#import train
import model
import utils
import evaluate as ev

# 📊 Cargar datos y entrenar modelo
train_gen, val_gen = train.get_data_generators(config)

cnn_model = model.build_model(config)

history = cnn_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=config["training"]["epochs"]
)

utils.save_model(cnn_model, config["paths"]["model"])

In [ ]:
# ---------------------------------------------
# 📈 Evaluación del modelo
# ---------------------------------------------

from tensorflow.keras.models import load_model

# 💾 Cargar modelo entrenado
model_path = config["paths"]["model"]
model = load_model(model_path)

# 🎯 Umbral configurable
threshold = config.get("evaluation", {}).get("threshold", 0.5)

# 🧪 Obtener predicciones y etiquetas verdaderas
y_true = val_gen.classes
y_prob = model.predict(val_gen).flatten()
y_pred = (y_prob > threshold).astype(int).flatten()

# 📈 Evaluación
ev.plot_confusion_matrix(y_true, y_pred)
ev.plot_roc_curve(y_true, y_prob)
ev.plot_prediction_distribution(y_true, y_prob)
ev.plot_f1_vs_threshold(y_true, y_prob)

In [ ]:
# ---------------------------------------------
# 🔍 Análisis de errores y Grad-CAM
# ---------------------------------------------

# Mostrar imágenes mal clasificadas
ev.show_misclassified_images(val_gen, cnn_model)